In [16]:
#chatgpt test
import prosail
import numpy as np
import matplotlib.pyplot as plt
import itertools
import csv
import time

In [31]:
# Define parameter ranges for a large mesh of possible combinations
N=100 #nummber of values for each IMPORTANT parameter
M=2 #number of values for each NON-IMPORTANT parameter
Q=2 #number of values for each AVERAGELY IMPORTANT parameter
n_values = [1.5] #list(np.linspace(1,3,M)) #[1.5, 1.6, 1.7]
cab_values = list(np.linspace(0,100,N))#[40] [40, 50, 60]
car_values = [8] #list(np.linspace(5,30,M)#[8, 10, 12]
cbrown_values = [0] # list(np.linspace(0,1,M))#[0]  TO CHECK!!!
cw_values = [0.01] #list(np.linspace(0.004,0.4,M)) #[0.01]#[0.01, 0.015, 0.02]
cm_values = [0.008] # list(np.linspace(0.0019,0.0165,M))#[0.008]#[0.008, 0.009, 0.01]
lai_values = list(np.linspace(0,8,N))#[3.0]#[3.0, 3.5, 4.0]
lidfa_values = list(np.linspace(0,90,N)) #[45]#[45, 50, 55]
hspot_values = [0.1] # list(np.linspace(0,1,M)) #[0.1]#[0.1, 0.2, 0.3]
tts_values = [20] # list(np.linspace(0,90,M)) #[20]#[20, 30, 40]
tto_values = [0] #list(np.linspace(0,90,M)) #[0]
psi_values = [0] #list(np.linspace(0,180,M)) #[0]
rsoil_values = list(np.linspace(0,5,Q)) #[1]
psoil_values = list(np.linspace(0,1,Q)) #[0.4, 0.5, 0.6]

# Generate all combinations of parameters
parameter_combinations = list(itertools.product(n_values, cab_values, car_values, cbrown_values, cw_values, cm_values, 
                                                lai_values, lidfa_values, hspot_values, tts_values, tto_values, 
                                                psi_values, rsoil_values, psoil_values))

# Wavelength range for PROSAIL output (from 400 nm to 2500 nm)
wavelengths = np.arange(400, 2501)

# Output file
output_file = '/home/yavor/Documents/sc/prosail_output/prosail_data_v1.csv'
i0=0 #beginning row, for easier restart in case of abnormal runs

In [32]:
t0=time.time()

# Open the output file and write header
with open(output_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    if i0==0:
        header = ['i','n', 'cab', 'car', 'cbrown', 'cw', 'cm', 'lai', 'lidfa', 'hspot', 'tts', 'tto', 'psi', 'rsoil', 'psoil'] + list(["B","G","R","NIR"])
        writer.writerow(header)

    # Iterate over all parameter combinations
    #for params in parameter_combinations:
    for i in range(i0,len(parameter_combinations)):
        params=parameter_combinations[i]
        n, cab, car, cbrown, cw, cm, lai, lidfa, hspot, tts, tto, psi, rsoil, psoil = params

        # Run the PROSAIL model with the current combination of parameters
        reflectance = prosail.run_prosail(n, cab, car, cbrown, cw, cm, lai, lidfa, hspot, tts, tto, psi,
                                          ant=0.0, alpha=40.0, prospect_version='5', typelidf=2, lidfb=0.0,
                                          factor='SDR', rsoil0=None, rsoil=rsoil, psoil=psoil,
                                          soil_spectrum1=None, soil_spectrum2=None)
        B=np.average(reflectance[58:122]) #blue, B02
        G=np.average(reflectance[143 : 177]) #green, B03
        R=np.average(reflectance[250 : 280]) #red, B04
        NIR=np.average(reflectance[386 : 500]) #nir, B08
        # Write the parameters and the corresponding reflectance to the file
        row = list([i])+list(params) + list([B,G,R,NIR])
        writer.writerow(row)
        print("Row: ",i," Time: ", time.time()-t0,end="\r")

print(f"Band results saved to {output_file}")

Band results saved to /home/yavor/Documents/sc/prosail_output/prosail_data_v1.csv
